# Train MNIST via Keras

- https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py

In [1]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import backend as K

D:\Program\miniconda3\envs\tf-1.12\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Program\miniconda3\envs\tf-1.12\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Program\miniconda3\envs\tf-1.12\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Program\miniconda3\envs\tf-1.12\lib\site-packages\tensorflow\python\frame

In [2]:
print("TensorFlow version %s" % tf.__version__)

TensorFlow version 1.12.0


In [3]:
batch_size = 128
num_classes = 10
epochs = 20

In [4]:
# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


- Run the cell below if "failed to create cublas handle: CUBLAS_STATUS_ALLOC_FAILED" error happens...

In [5]:
config = tf.ConfigProto()
# config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
# config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.Session(config=config)
tf.keras.backend.set_session(sess)  # set this TensorFlow session as the default session for Keras

In [6]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [7]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

## Training

In [8]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 8s 126us/step - loss: 0.2651 - acc: 0.9191 - val_loss: 0.0580 - val_acc: 0.9817
Epoch 2/20
60000/60000 [==============================] - 5s 78us/step - loss: 0.0873 - acc: 0.9733 - val_loss: 0.0448 - val_acc: 0.9849
Epoch 3/20
60000/60000 [==============================] - 5s 78us/step - loss: 0.0661 - acc: 0.9799 - val_loss: 0.0378 - val_acc: 0.9875
Epoch 4/20
60000/60000 [==============================] - 5s 79us/step - loss: 0.0543 - acc: 0.9839 - val_loss: 0.0314 - val_acc: 0.9897
Epoch 5/20
60000/60000 [==============================] - 5s 80us/step - loss: 0.0475 - acc: 0.9860 - val_loss: 0.0316 - val_acc: 0.9891
Epoch 6/20
60000/60000 [==============================] - 5s 79us/step - loss: 0.0416 - acc: 0.9878 - val_loss: 0.0281 - val_acc: 0.9905
Epoch 7/20
60000/60000 [==============================] - 5s 79us/step - loss: 0.0383 - acc: 0.9880 - val_loss: 0.0297 - val_ac

In [9]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.03118970596088875
Test accuracy: 0.9918


## Save model

In [10]:
model.save('lenet.h5')